In [1]:
import os 
import imageio
from IPython.display import display, Image
from sklearn.preprocessing import OneHotEncoder
import numpy as np 
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#for using in google colab
!unzip "drive/MyDrive/trainimages.zip" -d "/content/data/"

Streaming output truncated to the last 5000 lines.
 extracting: /content/data/train/images/go/94de6a6a_nohash_2.png  
 extracting: /content/data/train/images/go/34ba417a_nohash_0.png  
 extracting: /content/data/train/images/go/c0c0d87d_nohash_1.png  
 extracting: /content/data/train/images/go/f5c3de1b_nohash_1.png  
 extracting: /content/data/train/images/go/d0f7bef5_nohash_0.png  
 extracting: /content/data/train/images/go/8a28231e_nohash_0.png  
 extracting: /content/data/train/images/go/2313e093_nohash_1.png  
 extracting: /content/data/train/images/go/460f2bad_nohash_0.png  
 extracting: /content/data/train/images/go/1942abd7_nohash_0.png  
 extracting: /content/data/train/images/go/74b73f88_nohash_1.png  
 extracting: /content/data/train/images/go/ca48dc76_nohash_0.png  
 extracting: /content/data/train/images/go/e7ea8b76_nohash_5.png  
 extracting: /content/data/train/images/go/2fa39636_nohash_0.png  
 extracting: /content/data/train/images/go/acde7e73_nohash_1.png  
 extracting

In [4]:
train_path = 'data/train/images/'

#ignore .ipynb_checkpoints
classes = [f for f in os.listdir(train_path) if not f.startswith('.')]

train_x, train_y = [], []
#128x87 with 1 channel
image_shape = (128,87,1)

for c in classes:
  label = classes.index(c)
  print(label)
  class_path = train_path + c + '/'
  images = os.listdir(train_path+c+'/')
  for im in images:
    #display(Image(filename=class_path + im))
    x = np.array(imageio.imread(class_path+im))
    x = np.expand_dims(x,axis=0)
    x = np.resize(x,image_shape)
    train_x.append(x)
    train_y.append(label)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30


In [5]:
train_x = np.array(train_x)
train_y = np.array(train_y).reshape(-1,1)

onehot = OneHotEncoder()
train_y = onehot.fit_transform(train_y).toarray()
print(train_x.shape)
print(train_y.shape)
num_classes = train_y.shape[1]

(64727, 128, 87, 1)
(64727, 31)


In [6]:
shuffler = np.random.permutation(len(train_x))

train_x = train_x[shuffler]

train_y = train_y[shuffler]

In [7]:
model = keras.Sequential(
    [
        keras.Input(shape=image_shape),
        layers.Conv2D(64,kernel_size=(3,3),activation="relu",kernel_regularizer=regularizers.l2(4e-5)),
        layers.Conv2D(64,kernel_size=(3,3),activation="relu",kernel_regularizer=regularizers.l2(4e-5)),
        layers.Conv2D(64,kernel_size=(3,3),activation="relu",kernel_regularizer=regularizers.l2(4e-5)),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Conv2D(128,kernel_size=(3,3),activation="relu",kernel_regularizer=regularizers.l2(4e-5)),
        layers.Conv2D(128,kernel_size=(3,3),activation="relu",kernel_regularizer=regularizers.l2(4e-5)),
        layers.Conv2D(128,kernel_size=(3,3),activation="relu",kernel_regularizer=regularizers.l2(4e-5)),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Conv2D(256,kernel_size=(3,3),activation="relu",kernel_regularizer=regularizers.l2(4e-5)),
        layers.Conv2D(256,kernel_size=(3,3),activation="relu",kernel_regularizer=regularizers.l2(4e-5)),
        layers.Conv2D(256,kernel_size=(3,3),activation="relu",kernel_regularizer=regularizers.l2(4e-5)),
        layers.MaxPooling2D(pool_size=(2,2)),  
        layers.Flatten(),
        layers.Dense(512,activation="relu",kernel_regularizer=regularizers.l2(4e-5)),
        layers.Dense(num_classes,activation="softmax",kernel_regularizer=regularizers.l2(4e-5))
    ],
    name="cnn-9"
)

model.summary()

Model: "cnn-9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 85, 64)       640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 124, 83, 64)       36928     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 122, 81, 64)       36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 61, 40, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 59, 38, 128)       73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 57, 36, 128)       147584    
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 55, 34, 128)       147584

In [8]:
#opt = keras.optimizers.SGD(learning_rate = 0.01, momentum=0.9,nesterov=True)

opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt,loss="categorical_crossentropy",metrics=['accuracy'])

In [ ]:
#model = keras.models.load_model('weights.h5')
early_stop = EarlyStopping(monitor='val_loss',min_delta=.00001, patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5',monitor='val_accuracy', save_best_only=True,mode='max')
#model.fit(train_x,train_y,batch_size=50,epochs=100, callbacks=[early_stop,checkpoint], validation_split = 0.15)
model.fit(train_x,train_y,batch_size=100,epochs=100, validation_split = 0.1)

Epoch 1/100
583/583 [==============================] - 71s 89ms/step - loss: 11274.2572 - accuracy: 0.0348 - val_loss: 3.8038 - val_accuracy: 0.0383
Epoch 2/100
583/583 [==============================] - 49s 85ms/step - loss: 3.7876 - accuracy: 0.0349 - val_loss: 3.7576 - val_accuracy: 0.0377
Epoch 3/100
583/583 [==============================] - 49s 85ms/step - loss: 3.7525 - accuracy: 0.0350 - val_loss: 3.7342 - val_accuracy: 0.0383
Epoch 4/100
583/583 [==============================] - 49s 85ms/step - loss: 3.7308 - accuracy: 0.0356 - val_loss: 3.7164 - val_accuracy: 0.0377
Epoch 5/100
583/583 [==============================] - 49s 85ms/step - loss: 3.7131 - accuracy: 0.0352 - val_loss: 3.7026 - val_accuracy: 0.0335
Epoch 6/100
583/583 [==============================] - 49s 85ms/step - loss: 3.6986 - accuracy: 0.0372 - val_loss: 3.6875 - val_accuracy: 0.0400
Epoch 7/100
583/583 [==============================] - 49s 85ms/step - loss: 3.6859 - accuracy: 0.0364 - val_loss: 3.6752 - va

In [ ]:
model.save('weights.h5')